# SD-Turbo: Finding a reasonable CLAMP range for exploring the prompt embedding space
Using the diverse parti prompts v2 (1,6k entries) to explore the range of values for prompt embeddings. 

In [1]:
from evolutionary.prompt_encoding.image_creation import SDPromptEmbeddingImageCreator
from evolutionary.prompt_encoding.utils import clamp_range_from_parti
from model_helpers.auto_pipeline import auto_diffusion_pipeline

In [2]:
def setup_pipeline():
    pipe = auto_diffusion_pipeline("stabilityai/sd-turbo")
    pipe.set_progress_bar_config(disable=True)
    return pipe

In [3]:
creator = SDPromptEmbeddingImageCreator(pipeline_factory=setup_pipeline, batch_size=1, inference_steps=1)
test1 = clamp_range_from_parti(creator, lambda_accessor=lambda x: x.prompt_embeds)
print("Range for prompt_embeds: ", test1)

Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

Loaded StableDiffusionPipeline {
  "_class_name": "StableDiffusionPipeline",
  "_diffusers_version": "0.25.0",
  "_name_or_path": "stabilityai/sd-turbo",
  "feature_extractor": [
    null,
    null
  ],
  "image_encoder": [
    null,
    null
  ],
  "requires_safety_checker": false,
  "safety_checker": [
    null,
    null
  ],
  "scheduler": [
    "diffusers",
    "EulerDiscreteScheduler"
  ],
  "text_encoder": [
    "transformers",
    "CLIPTextModel"
  ],
  "tokenizer": [
    "transformers",
    "CLIPTokenizer"
  ],
  "unet": [
    "diffusers",
    "UNet2DConditionModel"
  ],
  "vae": [
    "diffusers",
    "AutoencoderKL"
  ]
}


Token indices sequence length is longer than the specified maximum sequence length for this model (84 > 77). Running this sequence through the model will result in indexing errors


The following part of your input was truncated because CLIP can only handle sequences up to 77 tokens: ['as a beacon over rolling blue hills']
Range for prompt_embeds:  (-10.234375, 15.6484375)


For the SD-Turbo model the CLAMP range for prompt_embeds is around (-10.3, 15.65).
Keep in mind you can extend the values, but this restricts the search space to a reasonable range.